In [ ]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("/kaggle/input/semeval-mar/mar_train.csv")
df.head()

In [ ]:
df1=pd.read_csv("/kaggle/input/semeval-mar/mar_dev.csv")
df1.head()

In [ ]:
df2=pd.read_csv("/kaggle/input/semeval-mar/mar_dev_with_labels.csv")
df2.head()

In [ ]:
df3=pd.read_csv("/kaggle/input/semeval-mar/mar_test.csv")
df3.head()

In [ ]:
df2['Text2'] = df2['Text'].apply(lambda x: x.split('\n')[1] if '\n' in x else '')
df2['Text2'] = df2['Text2'].str.replace('\n', '')
df2['Text1'] = df2['Text'].apply(lambda x: x.split('\n')[0] if '\n' in x else '')
df2['Text1'] = df2['Text1'].str.replace('\n', '')

In [ ]:
df3['Text2'] = df3['Text'].apply(lambda x: x.split('\n')[1] if '\n' in x else '')
df3['Text2'] = df3['Text2'].str.replace('\n', '')
df3['Text1'] = df3['Text'].apply(lambda x: x.split('\n')[0] if '\n' in x else '')
df3['Text1'] = df3['Text1'].str.replace('\n', '')

In [ ]:
df['Text2'] = df['Text'].apply(lambda x: x.split('\n')[1] if '\n' in x else '')
df['Text2'] = df['Text2'].str.replace('\n', '')
df['Text2'][0]

In [ ]:
df1['Text2'] = df1['Text'].apply(lambda x: x.split('\n')[1] if '\n' in x else '')
df1['Text2'] = df1['Text2'].str.replace('\n', '')


In [ ]:
df['Text1'] = df['Text'].apply(lambda x: x.split('\n')[0] if '\n' in x else '')
df['Text1'] = df['Text1'].str.replace('\n', '')
df['Text1'][0]

In [ ]:
df1['Text1'] = df1['Text'].apply(lambda x: x.split('\n')[0] if '\n' in x else '')
df1['Text1'] = df1['Text1'].str.replace('\n', '')


In [ ]:
df = df.drop(columns=['Text'])
df

In [ ]:
df

In [ ]:
import re
import string

In [ ]:
marathi_numerals_pattern = re.compile(r'[\u0966-\u096F]+')

# Function to remove Marathi numerals from a text
def remove_marathi_numerals(text):
    return marathi_numerals_pattern.sub('', text)

# Apply the function to the 'Text1' column
df['Text1'] = df['Text1'].apply(remove_marathi_numerals)
df['Text2'] = df['Text2'].apply(remove_marathi_numerals)
df1['Text1'] = df1['Text1'].apply(remove_marathi_numerals)
df1['Text2'] = df1['Text2'].apply(remove_marathi_numerals)
df2['Text1'] = df2['Text1'].apply(remove_marathi_numerals)
df2['Text2'] = df2['Text2'].apply(remove_marathi_numerals)
df3['Text1'] = df3['Text1'].apply(remove_marathi_numerals)
df3['Text2'] = df3['Text2'].apply(remove_marathi_numerals)

In [ ]:
spanish_alphabet_pattern = re.compile(r'\D+')

# Apply the pattern to the 'Text' column
df['Text1'] = df['Text1'].apply(lambda x: ''.join(spanish_alphabet_pattern.findall(x)))
df['Text2'] = df['Text2'].apply(lambda x: ''.join(spanish_alphabet_pattern.findall(x)))

df1['Text1'] = df1['Text1'].apply(lambda x: ''.join(spanish_alphabet_pattern.findall(x)))
df1['Text2'] = df1['Text2'].apply(lambda x: ''.join(spanish_alphabet_pattern.findall(x)))
df2['Text1'] = df2['Text1'].apply(lambda x: ''.join(spanish_alphabet_pattern.findall(x)))
df2['Text2'] = df2['Text2'].apply(lambda x: ''.join(spanish_alphabet_pattern.findall(x)))
df3['Text1'] = df3['Text1'].apply(lambda x: ''.join(spanish_alphabet_pattern.findall(x)))
df3['Text2'] = df3['Text2'].apply(lambda x: ''.join(spanish_alphabet_pattern.findall(x)))


In [ ]:
pip install pandas torch transformers


# **Transformer-1**

In [ ]:
pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util

In [ ]:
model = SentenceTransformer('l3cube-pune/marathi-sentence-similarity-sbert')

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)


In [ ]:
train_df=df

In [ ]:
val_df = df2

In [ ]:
from sentence_transformers import SentenceTransformer, SentencesDataset, LoggingHandler, losses

model = SentenceTransformer('l3cube-pune/marathi-sentence-similarity-sbert')

In [ ]:
from torch.utils.data import DataLoader
from sentence_transformers.readers import InputExample

train_examples = [InputExample(texts=[row['Text1'], row['Text2']], label=row['Score']) for _, row in train_df.iterrows()]
train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)

val_examples = [InputExample(texts=[row['Text1'], row['Text2']], label=row['Score']) for _, row in val_df.iterrows()]
val_dataset = SentencesDataset(val_examples, model)
val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=16)


In [ ]:
from torch.nn import BCEWithLogitsLoss

loss_function = BCEWithLogitsLoss()


In [ ]:

from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
import torch

# Initialize the model
model = SentenceTransformer('l3cube-pune/marathi-sentence-similarity-sbert')
# Define the cosine similarity loss function
loss_function = losses.CosineSimilarityLoss(model)

# Prepare the data
train_examples = [InputExample(texts=[row['Text1'], row['Text2']], label=row['Score']) for _, row in train_df.iterrows()]

# Create a DataLoader
train_dataloader = DataLoader(train_examples, batch_size=16, shuffle=True)

sentences1 = [example.texts[0] for example in val_examples]
sentences2 = [example.texts[1] for example in val_examples]
scores = [example.label for example in val_examples]

# Create an evaluator using cosine similarity
evaluator = EmbeddingSimilarityEvaluator(sentences1, sentences2, scores)

# Adjust warm-up steps according to your preference
warmup_steps = 750


model.fit(
    train_objectives=[(train_dataloader, loss_function)],
    epochs=10,
    evaluator=evaluator,
    optimizer_class= torch.optim.AdamW,
    optimizer_params={'lr': 9e-5},
    weight_decay=5e-2,  
    evaluation_steps=1000,
    output_path='model3',
    save_best_model=True,
    show_progress_bar=True,
    
)


In [ ]:
model = SentenceTransformer('model3')

def compute_cosine_similarity(row):
    embedding_1 = model.encode(row['Text1'], convert_to_tensor=True)
    embedding_2 = model.encode(row['Text2'], convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(embedding_1, embedding_2)
    return similarity.item()

# Create a new column 'CosineSimilarity' and apply the function
df2['CosineSimilarity'] = df2.apply(compute_cosine_similarity, axis=1)

# Display the updated DataFrame
# print(df)

In [ ]:
df2['CosineSimilarity'] = df2['CosineSimilarity'].apply(lambda x: 0 if x < 0 else x)
df2.head()

In [ ]:
from scipy.stats import spearmanr

# Assuming df is your DataFrame
correlation, _ = spearmanr(df2['Score'], df2['CosineSimilarity'])

print(f"Spearman Correlation: {correlation}")

In [ ]:
df3['CosineSimilarity'] = df3.apply(compute_cosine_similarity, axis=1)

In [ ]:
transformer_df = df3.drop(columns=['Text1', 'Text2','Text'])
transformer_df.head()

In [ ]:
transformer_df = transformer_df.rename(columns={'CosineSimilarity': 'Pred_Score'})


In [ ]:
transformer_df = df3.drop(columns=['Text1', 'Text2','Text'])
transformer_df.head()

In [ ]:
transformer_df = transformer_df.rename(columns={'CosineSimilarity': 'Pred_Score'})


In [ ]:
transformer_df['Pred_Score'] = transformer_df['Pred_Score'].apply(lambda x: 0 if x < 0 else x)
transformer_df.head()

In [ ]:
transformer_df.to_csv("pred_mar_a.csv",index=False)